In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import os


In [2]:
#similar images
image1_path = './data/images/GB.292020.A-009.tif'
image2_path = './data/images/GB.191319946.A-006.tif'
image3_path = './data/images/GB.425576.A-003.tif'
image4_path = '/Users/mac/Desktop/tbi/Semester-Project/data/images/GB.512507.A-003.tif'

In [3]:


def is_contour_inside(contour1, contour2):
    x1, y1, w1, h1 = cv2.boundingRect(contour1)  # inside
    x2, y2, w2, h2 = cv2.boundingRect(contour2)  # outside
    if x1 >= x2 and y1 >= y2 and x1 + w1 <= x2 + w2 and y1 + h1 <= y2 + h2:
        return True

def bounding_box_to_contour(box):
    x, y, w, h = box
    contour = np.array([
        [x, y],
        [x + w, y],
        [x + w, y + h],
        [x, y + h]
    ])
    return contour

In [4]:
gb = []
for i in os.listdir('./data/images'):
    if i.startswith('GB'):
        gb.append(i)
gb

['GB.292020.A-009.tif',
 'GB.135907.A-009.tif',
 'GB.112704.A-011.tif',
 'GB.259436.A-004.tif',
 'GB.191311226.A-003.tif',
 'GB.419964.A-010.tif',
 'GB.563092.A-006.tif',
 'GB.136290.A-005.tif',
 'GB.179134.A-003.tif',
 'GB.357706.A-006.tif',
 'GB.426870.A-007.tif',
 'GB.269702.A-004.tif',
 'GB.220420.A-010.tif',
 'GB.191216608.A-005.tif',
 'GB.281543.A-009.tif',
 'GB.472610.A-004.tif',
 'GB.424780.A-008.tif',
 'GB.191407918.A-006.tif',
 'GB.410169.A-005.tif',
 'GB.456294.A-006.tif',
 'GB.234354.A-005.tif',
 'GB.394336.A-006.tif',
 'GB.544722.A-035.tif',
 'GB.544722.A-021.tif',
 'GB.358665.A-003.tif',
 'GB.411884.A-027.tif',
 'GB.190921295.A-004.tif',
 'GB.188761.A-004.tif',
 'GB.411884.A-033.tif',
 'GB.503320.A-020.tif',
 'GB.191227946.A-004.tif',
 'GB.411963.A-008.tif',
 'GB.175414.A-004.tif',
 'GB.405924.A-003.tif',
 'GB.115043.A-004.tif',
 'GB.496119.A-009.tif',
 'GB.252432.A-008.tif',
 'GB.272628.A-015.tif',
 'GB.191506686.A-011.tif',
 'GB.545196.A-008.tif',
 'GB.185698.A-003.tif'

In [5]:
def process(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_canny = cv2.Canny(img_gray, 0, 150)
    kernel = np.ones((5, 5), np.uint8)
    img_dilate = cv2.dilate(img_canny, kernel, iterations=3)
    return img_dilate

#first phase of segmentation to remove outer border present in images
def first_phase(img_path):
    img = cv2.imread(img_path)
    img_copy = np.copy(img)
    
    processed_img = process(img_copy)
    contours, hierarchy = cv2.findContours(processed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filter_contours = [ct for ct in contours if cv2.contourArea(ct) > 3000] #filter out small contours
    areas = []
    original_size = img_copy.shape
    for ct in filter_contours:
        x, y, w, h = cv2.boundingRect(ct)
        areas.append(w*h)
    biggest = filter_contours[areas.index(max(areas))]
    oldx, oldy, oldw, oldh = cv2.boundingRect(biggest)
    #paddingx= oldw* 1//30
    #paddingy= oldh* 1//20
    paddingx= 100
    paddingy= 100
    new_w = oldw - paddingx
    new_h = oldh - paddingy
    new_x = oldx + paddingx
    new_y = oldy + paddingy
    biggest = img_copy[new_y:oldy+new_h, new_x:oldx+new_w]
    # Create a white background image with the same size as the original
    white_background = np.ones((original_size[0], original_size[1], 3), dtype=np.uint8) * 255
    
    # Calculate the center position
    #center_x = (original_size[1] - biggest.shape[1]) // 2
    #center_y = (original_size[0] - biggest.shape[0]) // 2
    
    # Place the ROI in the center of the white background
    white_background[new_y:new_y+biggest.shape[0], new_x:new_x+biggest.shape[1]] = biggest
    #for ct in filter_contours:
        #x, y, w, h = cv2.boundingRect(ct)
        #cv2.rectangle(img_copy, (x+70, y+70), (x + w-50, y + h-50), (0, 0, 255), 2)
    return img_copy, filter_contours,white_background
        
    
def second_phase(img_first_phase,img_original):
    img_copy = np.copy(img_first_phase)
    
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(process(img_copy), connectivity=8, ltype=cv2.CV_32S)
    contours = []
    #area = img_copy.shape[0] * img_copy.shape[1]
    for i in range(1, num_labels):
        x, y, w, h, _ = stats[i]
        box = (x, y, w, h)
        contour = bounding_box_to_contour(box)
        contours.append(contour)
        
    filtered_contours = [contour for contour in contours if cv2.contourArea(contour) > 3000]
    # get rid of small contours
    sorted_contours = sorted(filtered_contours, key=cv2.contourArea, reverse=True)
    
        
    for idx,contour in enumerate(sorted_contours):
        if idx == 0:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(img_original, (x, y), (x + w, y + h), (0, 0, 255), 2)
        else:
            #x, y, w, h = cv2.boundingRect(contour)
            res = False
            for i in range(idx):
                if is_contour_inside(contour, sorted_contours[i]):
                    res = True
                    break
            if not res: #the contour is not inside any of the previous contours
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(img_original, (x, y), (x + w, y + h), (0, 0, 255), 2)
                


    return img_copy, sorted_contours,img_original
    
    
img_copy, _,figure = first_phase(image3_path)
cv2.imshow("phase 2", figure)
cv2.waitKey(0)
cv2.destroyAllWindows()
_, _,segmented = second_phase(figure,img_copy)
cv2.imshow("phase 2", segmented)
cv2.waitKey(0)
cv2.destroyAllWindows()




In [86]:
for i in gb[20:30]:
    img_copy, _,figure = first_phase("./data/images/"+i)
    cv2.imshow("phase 1", figure)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    _, _,segmented = second_phase(figure,img_copy)
    cv2.imshow("phase 2", segmented)
    cv2.waitKey(0)
    cv2.destroyAllWindows()